In [31]:
import yalefaceDataset as yfd

In [32]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.autograd import Variable
from sklearn.preprocessing import LabelEncoder

In [40]:
yaleface_data = yfd.YalefaceDataset(annotations_file = "./data/yaleface.csv",\
                                    img_dir = "./data/clean_yalefaces/",\
                                    transform=ToTensor())

In [41]:
data_loader = DataLoader(dataset=yaleface_data,
                                            batch_size=10,
                                            shuffle=False)

In [42]:
for i,(images,labels) in enumerate(data_loader):
    print(1)

IndexError: string index out of range